In [1]:
import os
import sys
sys.path.append('/Users/nvalett/Documents/Natalie/Species Dist Research/Thesis/EcoLinker')
from EcoLinker.restorationOptimizer import restorationOptimizer
import ecoscape_connectivity

DATA_PATH="/Users/nvalett/Documents/Natalie/Species Dist Research/Thesis/EcoLinker/tests/assets/paper_repro"

HABITAT_PATH = os.path.join(DATA_PATH, "habitat.tif")
TERRAIN_PATH = os.path.join(DATA_PATH, "terrain.tif")
PERMEABILITY_PATH = os.path.join(DATA_PATH, "transmission_refined_1.csv")
transmission_d = ecoscape_connectivity.util.read_transmission_csv(PERMEABILITY_PATH)

CONNECTIVITY_PATH = os.path.join(DATA_PATH, "repopulation_spreads_15_hop_3_sims_10000_texp_1.tif")
FLOW_PATH = os.path.join(DATA_PATH, "gradient_spreads_15_hop_3_sims_1000_texp_1.tif")

In [2]:
from scgt import Tile, GeoTiff
import numpy as np

optimizer = restorationOptimizer(HABITAT_PATH, TERRAIN_PATH, CONNECTIVITY_PATH, FLOW_PATH, PERMEABILITY_PATH, 10)

death_tif = optimizer.get_death_layer("death.tif")
# print("death:")
# death_tif.draw_geotiff()

death_matrix = death_tif.get_all_as_tile().m.squeeze(0)
flat_indices = np.argpartition(death_matrix.ravel(), -optimizer.pixels)[-optimizer.pixels:]
row_indices, col_indices = np.unravel_index(flat_indices, death_matrix.shape)

min_elements = death_matrix[row_indices, col_indices]
min_elements_order = np.argsort(min_elements)
row_indices, col_indices = row_indices[min_elements_order], col_indices[min_elements_order]

for i in range(optimizer.pixels):
    print(f"death at pixel x={col_indices[i]}, y={row_indices[i]}: {death_matrix[row_indices[i]][col_indices[i]]}")

highest_death = optimizer.get_highest_death_pixels(death_tif)
print(highest_death)

death at pixel x=1146, y=2291: 89
death at pixel x=2224, y=4067: 90
death at pixel x=1907, y=4085: 90
death at pixel x=2225, y=4067: 91
death at pixel x=2007, y=4152: 91
death at pixel x=1910, y=4087: 92
death at pixel x=2226, y=4068: 92
death at pixel x=1043, y=2246: 94
death at pixel x=1036, y=2245: 96
death at pixel x=1755, y=3538: 99
{(1755, 3538): 99, (1036, 2245): 96, (1043, 2246): 94, (2226, 4068): 92, (1910, 4087): 92, (2007, 4152): 91, (2225, 4067): 91, (1907, 4085): 90, (2224, 4067): 90}
